# Preparation


Install the necessary packages:


In [ ]:
! pip install torch weaver-core matplotlib


In [ ]:
import os
import numpy as np
import awkward as ak
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import vector
vector.register_awkward()


Here defines some helper functions to visualize a jet:


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

typelist = ['ch+', 'ch-', 'nh', 'ph', 'el+', 'el-', 'mu+', 'mu-']


def make_subplot(ax, data, force_xylim=None):
    # default plotting configuration
    color_dict_ = {'ch': 'C0', 'nh': 'mediumpurple', 'ph': 'orange', 'el': 'red', 'mu': 'green'}
    color_dict = color_dict_.copy()
    color_dict.update({k + '+': color_dict_[k] for k in color_dict_})
    color_dict.update({k + '-': color_dict_[k] for k in color_dict_})
    if data.get('id') is None:
        data['id'] = ['default'] * len(data['pt'])
    if data.get('e') is None:
        for eta, phi, pt, id, d3d in zip(data['eta'], data['phi'], data['pt'], data['id'], data['d3d']):
            ptdraw = np.sqrt(pt) / 200
            alpha = 0.3
            if id in [4, 5]:
                ax.add_patch(mpl.patches.RegularPolygon((eta, phi), 3, radius=ptdraw, clip_on=True,
                                                        alpha=alpha, edgecolor='black', **make_color_args(id, d3d)))
            elif id in [6, 7]:
                ax.add_patch(mpl.patches.RegularPolygon((eta, phi), 3, radius=ptdraw, orientation=np.pi,
                                                        clip_on=True, alpha=alpha, edgecolor='black', **make_color_args(id, d3d)))
            elif id in [3]:
                ax.add_patch(mpl.patches.RegularPolygon((eta, phi), 5, radius=ptdraw,
                                                        clip_on=True, alpha=alpha, **make_color_args(id, d3d)))
            else:
                ax.add_patch(plt.Circle((eta, phi), ptdraw, clip_on=True, alpha=alpha, **make_color_args(id, d3d)))
    else:
        for eta, phi, pt, e, id, d3d in zip(data['eta'], data['phi'], data['pt'], data['e'], data['id'], data['d3d']):
            ax.add_patch(mpl.patches.Wedge((eta, phi), pt / 600., 90, 270,
                                           clip_on=True, alpha=alpha, **make_color_args(id, d3d)))
            ax.add_patch(mpl.patches.Wedge((eta, phi), e / 600., 270, 90,
                                           clip_on=True, alpha=alpha, **make_color_args(id, d3d)))
    max_ang = force_xylim if force_xylim else max(max(abs(data['eta'])), max(abs(data['phi'])))
    # make square plot centered at (0,0)
    ax.set_xlim(-max_ang, max_ang)
    ax.set_ylim(-max_ang, max_ang)
    ax.set_xlabel(r'$\Delta\eta$')
    ax.set_ylabel(r'$\Delta\phi$')
    ax.set_aspect('equal')
    return max_ang


def make_color_args(id, d3d):
    color = color_fader('#74c476', '#081d58', d3d)
    if id in [2, 3]:
        return {'edgecolor': color, 'linewidth': 1, 'fill': False}
    else:
        return {'facecolor': color}


def color_fader(c1, c2, mix=0):  # fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    mix = min(1., mix)
    c1 = np.array(mpl.colors.to_rgb(c1))
    c2 = np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1 - mix) * c1 + mix * c2)


def visualize(arrays, idx=0, title=None, ax=None):
    data = {}
    data['pt'] = np.hypot(arrays[idx].part_px, arrays[idx].part_py)
    data['eta'] = arrays[idx].part_deta
    data['phi'] = arrays[idx].part_dphi
    data['d3d'] = np.tanh(np.hypot(arrays[idx].part_d0val, arrays[idx].part_dzval))
    part_type = np.concatenate([
        [(arrays[idx].part_isChargedHadron) & (arrays[idx].part_charge == 1)],
        [(arrays[idx].part_isChargedHadron) & (arrays[idx].part_charge == -1)],
        [arrays[idx].part_isNeutralHadron],
        [arrays[idx].part_isPhoton],
        [(arrays[idx].part_isElectron) & (arrays[idx].part_charge == 1)],
        [(arrays[idx].part_isElectron) & (arrays[idx].part_charge == -1)],
        [(arrays[idx].part_isMuon) & (arrays[idx].part_charge == 1)],
        [(arrays[idx].part_isMuon) & (arrays[idx].part_charge == -1)],
    ], axis=0)
    data['id'] = np.argmax(part_type.T, axis=1)  # better

    assert len(data['eta'] == data['id'])
    if ax is None:
        _, ax = plt.subplots(figsize=(5, 5))
    make_subplot(ax, data, force_xylim=0.5)
    if title:
        ax.set_title(title)
    return ax


# Download the dataset


In [ ]:
def download(url, fname, chunk_size=1024):
    '''https://gist.github.com/yanqd0/c13ed29e29432e3cf3e7c38467f42f51'''
    import requests
    from tqdm import tqdm

    if os.path.dirname(fname):
        os.makedirs(os.path.dirname(fname), exist_ok=True)

    resp = requests.get(url, stream=True)
    total = int(resp.headers.get('content-length', 0))
    with open(fname, 'wb') as file, tqdm(
        desc=fname,
        total=total,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in resp.iter_content(chunk_size=chunk_size):
            size = file.write(data)
            bar.update(size)


In [ ]:
signal_file = './JetClassMini/TTBar_000.root'
background_file = './JetClassMini/ZJetsToNuNu_000.root'

if not os.path.exists(signal_file):
    download('https://hqu.web.cern.ch/datasets/JetClassMini/TTBar_000.root', signal_file)
if not os.path.exists(background_file):
    download('https://hqu.web.cern.ch/datasets/JetClassMini/ZJetsToNuNu_000.root', background_file)


# Train a light-weight ParticleNet model


For the training of more complex networks, we are going to use [weaver](https://github.com/hqucms/weaver-core) to handles all the data loading, preprocessing, and the boilerplate PyTorch training code. 

To train a neural network using `weaver`, we just need to use the CLI and specify two configuration files:

- A YAML _data configuration file_ describing how to process the input data.
- A python _model configuration file_ providing the neural network module and the loss function.


For this tutorial, we are going to use the configurations from this github repo:

In [ ]:
! git clone https://github.com/hqucms/dl4hep.git -b MITP2023

Let's have a look at the data config first: [particle_cloud/data/JetClassMini.yaml](https://github.com/hqucms/dl4hep/blob/MITP2023/particle_cloud/data/JetClassMini.yaml)

In [ ]:
! cat dl4hep/particle_cloud/data/JetClassMini.yaml

And the network config: [particle_cloud/networks/example_ParticleNet.py](https://github.com/hqucms/dl4hep/blob/MITP2023/particle_cloud/networks/example_ParticleNet.py)

In [ ]:
! cat dl4hep/particle_cloud/networks/example_ParticleNet.py

The following will run the training of a light-weight ParticleNet model, and then apply the training to the test dataset. The prediction scores will be saved to two files in the current directory:

- pred_particle_net_TTBar.root
- pred_particle_net_ZJetsToNuNu.root


In [ ]:
! weaver \
    --data-train "TTBar:./JetClassMini/TTBar_*.root" "ZJetsToNuNu:./JetClassMini/ZJetsToNuNu_*.root" \
    --data-test "TTBar:./JetClassMini/TTBar_*.root" "ZJetsToNuNu:./JetClassMini/ZJetsToNuNu_*.root" \
    --data-config dl4hep/particle_cloud/data/JetClassMini.yaml \
    --network-config dl4hep/particle_cloud/networks/example_ParticleNet.py -o conv_params "[(7,(32,32,32)),(7,(64,64,64))]" \
    --model-prefix "training/JetClassMini/PN/{auto}/net" \
    --num-workers 0 --fetch-step 1 --batch-size 512 --start-lr 1e-2 --num-epochs 20 \
    --optimizer ranger --log "logs/JetClassMini_PN_{auto}.log" --predict-output ./pred_particle_net.root \
    --tensorboard JetClassMini_ParticleNet \
    --gpus 0


# Evaluated the performance of the trained model


In [ ]:
from sklearn.metrics import accuracy_score, roc_curve, auc
from scipy.interpolate import interp1d


In [ ]:
df_test_signal = uproot.open("pred_particle_net_TTBar.root")["Events"].arrays(library="pd")
df_test_background = uproot.open("pred_particle_net_ZJetsToNuNu.root")["Events"].arrays(library="pd")
df_test = pd.concat([df_test_signal, df_test_background])
df_test


In [ ]:
# Truth label: Top jet=1, q/g jet=0
y_true = df_test["label_Tbqq"]


In [ ]:
prediction_probs = df_test[["score_label_QCD", "score_label_Tbqq"]].values
prediction_probs


In [ ]:
plt.figure(figsize=(6, 6))
plt.hist([prediction_probs[y_true == 1, 1], prediction_probs[y_true == 0, 1]],
         label=["Top", "q/g"], bins=50, histtype="step", density=True)
plt.yscale("log")
plt.xlabel("Prediction probability")
plt.legend()


In [ ]:
# fpr = epsilon_B, tpr = epsilon_S
fpr, tpr, thresholds = roc_curve(y_true=y_true, y_score=prediction_probs[:, 1])
auc_test = auc(fpr, tpr)

plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, label=f"XGBoost (AUC={auc_test:.4f})")
plt.plot([0, 1], [0, 1], ls="--", color="k")
plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
plt.legend()


In [ ]:
prediction_class = prediction_probs.argmax(1)
prediction_class


In [ ]:
acc = accuracy_score(df_test["label_Tbqq"], prediction_class)
print(f"Accuracy: {acc}")


In [ ]:
background_eff_fn = interp1d(tpr, fpr)
background_eff_at_50 = background_eff_fn(0.5)
print(f"Backround rejection at signal efficiency 50%: {1/background_eff_at_50:.0f}")


In [ ]:
def compute_metrics(y_true, probs):
    fpr, tpr, _ = roc_curve(y_true=y_true, y_score=probs[:, 1])
    auc_test = auc(fpr, tpr)
    acc_test = accuracy_score(y_true, probs.argmax(1))
    background_eff_fn = interp1d(tpr, fpr)
    background_eff_at_50 = background_eff_fn(0.5)

    print(f"Accuracy: {acc_test:.4f}")
    print(f"AUC: {auc_test:.4f}")
    print(f"Backround rejection at 50% signal efficiency: {1/background_eff_at_50:.0f}")
    return fpr, tpr


In [ ]:
compute_metrics(y_true, prediction_probs)


# Visualize the results


In [ ]:
signal_table = uproot.open(signal_file)["tree"].arrays()
background_table = uproot.open(background_file)["tree"].arrays()


In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(25, 10))
for idx in range(10):
    prob = prediction_probs[idx, 1]
    visualize(signal_table, idx, title=f"Top quark jet {idx}, prob(Top)={prob:.4f}", ax=axes[idx % 2][idx // 2])


In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(25, 10))
for idx in range(10):
    prob = prediction_probs[idx + 20000, 1]
    visualize(background_table, idx, title=f"q/g jet {idx}, prob(Top)={prob:.4f}", ax=axes[idx % 2][idx // 2])
